<a href="https://colab.research.google.com/drive/1fE3QxNXibveRcVqjK3HQ9EiDUKebD3NQ?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CPU Parallelism: Vector Processing and Multithreading

In this notebook, we'll explore how you can achieve parallel execution on CPUs by way of vector processing and multi-threading, using `numba`. Here, we'll focus on incorporating `numba` into a common analytical workflow -- that of applying some function to a column (or several columns) in your DataFrame and creating a new, derived column for further study.

For this demonstration, we'll be working with a small sample of [AirBnB's listing data](http://insideairbnb.com/get-the-data.html), a large dataset that contains information on AirBnBs from around the world on a month-by-month basis.

In [1]:
from numba.pycc import CC
from numba import vectorize, jit, prange
import numpy as np
import pandas as pd

Then, we can load in our AirBnB data (included in this directory) and see what it looks like (note that this data is from the city of Chicago, compiled by AirBnB in April 2021):


In [2]:
df = pd.read_csv('listings_chi.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,65,2,182,2021-03-28,2.38,1,0
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85373,-87.69540,Entire home/apt,113,2,395,2020-07-14,2.67,1,180
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90166,-87.68021,Entire home/apt,65,2,394,2021-04-11,2.74,1,267
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91943,-87.63898,Entire home/apt,120,5,54,2021-01-15,0.63,11,1
4,10945,The Biddle House (#1),33004,At Home Inn,NaN,Lincoln Park,41.91196,-87.63981,Entire home/apt,175,4,22,2021-03-25,0.26,11,125


You'll notice that two of the columns in the DataFrame are "latitude" and "longitude" -- spatial coordinates corresponding to AirBnB locations. Let's say that we're interested in creating a derived column from these coordinates, measuring how far each AirBnB is from the MACSS building at the University of Chicago (so that we can compute some further summary statistics about this column). The longitude and latitude of the 1155 E. 60th Street building is as follows:

In [3]:
macss = {'longitude': -87.5970978, 'latitude': 41.7856443}


To measure distance between coordinates, we can write a Python function to calculate the distance between two sets of (longitude, latitude) coordinates using [great-circle distance](https://en.wikipedia.org/wiki/Great-circle_distance). We'll write another version of this function that uses `numba` to compile this function ahead of time in two ways: one that performs the calcuation on individual scalar values and another that does so via vector processing.

In [4]:
def distance(lon1, lat1, lon2, lat2):
    '''                                                                         
    Calculate the circle distance between two points                            
    on the earth (specified in decimal degrees)                                 
    '''
    # convert decimal degrees to radians                                        
    lon1, lat1 = map(np.radians, [lon1, lat1])
    lon2, lat2 = map(np.radians, [lon2, lat2])

    # haversine formula                                                         
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # 6367 km is the radius of the Earth                                        
    km = 6367 * c
    m = km * 1000
    return m

# Use Numba to compile this same function in a module named `aot`
# in both a vectorized and non-vectorized form
cc = CC('aot')

@cc.export('distance', 'f8(f8,f8,f8,f8)')
@cc.export('distance_v', 'f8[:](f8[:],f8[:],f8,f8)')
def distance_numba(lon1, lat1, lon2, lat2):
    '''                                                                         
    Calculate the circle distance between two points                            
    on the earth (specified in decimal degrees)
    
    (distance: Numba-accelerated; distance_v: Numba-accelerated + vectorized)
    '''
    # convert decimal degrees to radians                        
    lon1, lat1 = map(np.radians, [lon1, lat1])
    lon2, lat2 = map(np.radians, [lon2, lat2])

    # haversine formula                                                         
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # 6367 km is the radius of the Earth                                        
    km = 6367 * c
    m = km * 1000
    return m
cc.compile()

import aot # import in module we just compiled

First, we could use the common dataframe method `apply` to apply our function to each row in our dataframe. Note, though, that this is really slow and is effectively equivalent to looping over each row in the dataframe using our pre-compiled distance function.

In [5]:
%%timeit
df.loc[:,'distance_from_macss'] = df[['longitude', 'latitude']] \
                                    .apply(lambda x: distance(x.longitude,
                                                              x.latitude,
                                                              macss['longitude'],
                                                              macss['latitude']),
                                           axis=1)

508 ms ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
df.loc[:,'distance_from_macss'] = df[['longitude', 'latitude']] \
                                    .apply(lambda x: aot.distance(
                                        x.longitude,
                                        x.latitude,
                                        macss['longitude'],
                                        macss['latitude']),
                                      axis=1)
# faster looping, but still looping

144 ms ± 27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
distance_lst = []
for lon, lat in df[['longitude', 'latitude']].values:
  dist = distance(lon,
                  lat,
                  macss['longitude'],
                  macss['latitude'])
  distance_lst.append(dist)
df.loc[:,'distance_from_macss'] = distance_lst
# see that this solution is essentially the same!

132 ms ± 34.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


When you're working with `numpy` or `pandas` arrays it's generally better to to take advantage of their capacity for parallel vector processing (up to the capabilities of your CPU). The built-in `numpy` functions in our distance function, for instance, will all take both scalars and vectors as input and be able to perform vectorized operations on multiple elements of our arrays in parallel. For instance, even without compiling our code with `numba`, we see ~50x speedup simply by using existing `numpy` functionality:

In [8]:
%%timeit
df.loc[:,'distance_from_macss'] = distance(df.longitude, 
                                           df.latitude,                  
                                           macss['longitude'],
                                           macss['latitude'])

2.57 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Our `numba` pre-compiled solution is even faster, for the reasons discussed last week:

In [9]:
%%timeit
# note `.values`! Numba doesn't accept Pandas objects (but NumPy is OK)
df.loc[:,'distance_from_macss'] = aot.distance_v(df.longitude.values, 
                                                 df.latitude.values,                  
                                                 macss['longitude'],
                                                 macss['latitude'])

703 µs ± 39.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


As mentioned above, the main reason for our vector processing speedup is that our computation involved a lot of predefined NumPy functions that are already compiled for us to perform these operations via vector processing.

Note, for instance that `np.sin` (one of the functions we used) is a "Universal Function" (`ufunc`):

In [10]:
type(np.sin)

numpy.ufunc

If we want to compile our own `ufunc`s to perform vectorized operations on NumPy arrays, `numba` includes a `@vectorize` decorator as well that will compile a `ufunc` for us. This can be useful if our particular application is not supported by the existing set of `numpy` universal function.

Let's write a `ufunc`, for instance, that checks a variety of conditions in our dataset and assigns categorical labels to postings based on whether they match the expected conditions or not:

In [11]:
@vectorize(['i8(i8,i8)'])
def check_conditions_v(id, number_of_reviews):
    check_lst = set([i for i in range(5700, 5800)])
    if (id in check_lst) and (number_of_reviews > 50):
      return 0
    elif (number_of_reviews > 50):
      return 1
    return 2

df.loc[:, 'condition'] = check_conditions_v(df.host_id, df.number_of_reviews)
df.loc[:5, ['host_id', 'number_of_reviews', 'condition']]

,host_id,number_of_reviews,condition
0,2613,182,1
1,5775,395,0
2,17928,394,1
3,33004,54,1
4,33004,22,2
5,40731,9,2


## "Multithreading" in `numba`

`numba` allows us to achieve additional parallelism via optional multithreading capabilities. In Colab, we can check and see that we do have access to a minimal amount of parallelism via multithreading (two threads!) to demonstrate:

In [12]:
from numba import get_num_threads
get_num_threads()

2

In the context of vector processing, we can split up our input array into smaller arrays which are assigned to different threads to execute in parallel. This can be achieved simply by setting our compilation target to 'parallel' in the `vectorize` decorator:

In [13]:
@vectorize(['i8(i8,i8)'], target='parallel')
def check_conditions_v_mt(id, number_of_reviews):
    check_lst = set([i for i in range(5700, 5800)])
    if (id in check_lst) and (number_of_reviews > 50):
      return 0
    elif (number_of_reviews > 50):
      return 1
    return 2

Let's test to see how our vectorized version of the function compares to our vectorized + multithreaded version (increasing data size so that the difference is more distinguishable):

In [14]:
# increase data size 
host_ids = np.tile(df.host_id.values, 200)
num_reviews = np.tile(df.number_of_reviews.values, 200)

# compare original runtime
%timeit check_conditions_v(host_ids, num_reviews)
# to multi-threading version
%timeit check_conditions_v_mt(host_ids, num_reviews)

2.21 s ± 439 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.98 s ± 415 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



We see a slight speedup even here in Colab, but we are working on limited resources, so it is not too dramatic. However, working on 8 threads (2 threads per physical CPU core) on my local machine, you can see a significant performance boost via multithreading:

```ipython
In [38]: # compare original runtime
    ...: %timeit check_conditions_v(host_ids, num_reviews)
    ...: # to multi-threading version
    ...: %timeit check_conditions_v_mt(host_ids, num_reviews)
1.27 s ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
566 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

Note that we can also use `numba` to automatically parallelize loops for us via multithreading. Let's assume, for instance, that we are working on a dataset with over 60m records (increasing the size of our dataset again to more easily distinguish performance differences across versions of our functions) and we want to (log) transform all of the distances we've computed and then compute the average distance of all postings from the MACSS building.

In `numpy`, we could compute this statistic like so:

In [15]:
distances = np.tile(df.distance_from_macss.values, 10000)
len(distances)

63860000

In [16]:
%timeit np.log(distances).sum() / len(distances)

853 ms ± 226 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The time it takes to run this code is similar to compiling a function to do the same in `nopython` mode in `numba` (utilizing vector processing on one thread):

In [17]:
@jit(nopython=True)
def average_log_distance_from_macss(distance_from_macss):
  size = len(distance_from_macss)
  transformed_dist = np.log(distance_from_macss)
  sum_d = 0

  for i in range(size):
    sum_d += transformed_dist[i]
  
  avg = sum_d / size
  return avg

%timeit average_log_distance_from_macss(distances)

924 ms ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


`numba` can parallelize such functions for us by setting `parallel=True` and replacing `range` in our loop definition with `prange` to explicitly parallelize our loop iterations across available threads:

In [23]:
@jit(nopython=True, parallel=True)
def average_log_distance_from_macss_mt(distance_from_macss):
  size = len(distance_from_macss)
  transformed_dist = np.log(distance_from_macss)
  sum_d = 0

  # note the use `prange` instead of `range` to explicitly parallelize loop
  # across threads
  for i in prange(size):
    sum_d += transformed_dist[i]
  
  avg = sum_d / size
  return avg

%timeit average_log_distance_from_macss_mt(distances)

542 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


...resulting in a speedup even over the original `numpy` solution. If we use the built-in parallel diagnostics method for our function, we can see that `numba` achieves this speedup by inferring which steps in our code can be performed together and "fusing" the parallel code together for execution:

In [19]:
average_log_distance_from_macss_mt.parallel_diagnostics()

 
 Parallel Accelerator Optimizing:  Function average_log_distance_from_macss_mt, 
<ipython-input-18-2d539ca9ec92> (1)  


Parallel loop listing for  Function average_log_distance_from_macss_mt, <ipython-input-18-2d539ca9ec92> (1) 
-------------------------------------------------------------------------------|loop #ID
@jit(nopython=True, parallel=True)                                             | 
def average_log_distance_from_macss_mt(distance_from_macss):                   | 
  size = len(distance_from_macss)                                              | 
  transformed_dist = np.log(distance_from_macss)-------------------------------| #0
  sum_d = 0                                                                    | 
                                                                               | 
  # note the use `prange` instead of `range` to explicitly parallelize loop    | 
  # across threads                                                             | 
  for i in prange(siz